Egy haladó megoldás:

- Declare ( @@ global variable, @variable)
- Cursor
- while loop
- Print

In [1]:
USE sql_recap;

DECLARE @schema_name NVARCHAR(128) = 'dbo'; -- Állítsd be a kívánt sémát
DECLARE @table_name NVARCHAR(128);
DECLARE @row_count INT;
DECLARE @sql NVARCHAR(MAX);

-- Táblák lekérdezése a kívánt sémában
DECLARE table_cursor CURSOR FOR
SELECT table_name
FROM information_schema.tables
WHERE table_schema = @schema_name
  AND table_type = 'BASE TABLE';

-- A kurzor megnyitása
OPEN table_cursor;
-- Kurzor bejárása (soronként)
FETCH NEXT FROM table_cursor INTO @table_name;

-- WHILE ciklus, amíg van még tábla a kurzorban
WHILE @@FETCH_STATUS = 0
BEGIN
    -- Dinamikus SQL készítése, hogy lekérdezd a sorok számát
    SET @sql = 'SELECT @row_count_out = COUNT(*) FROM ' + QUOTENAME(@schema_name) + '.' + QUOTENAME(@table_name);

    -- A dinamikus SQL végrehajtása
    EXEC sp_executesql @sql, N'@row_count_out INT OUTPUT', @row_count OUTPUT;

    -- Az eredmény kiírása
    PRINT 'Table: ' + @table_name + ', Row Count: ' + CAST(@row_count AS NVARCHAR(20));

    -- Következő tábla lekérdezése
    FETCH NEXT FROM table_cursor INTO @table_name;
END

-- A kurzor bezárása és felszabadítása
CLOSE table_cursor;
DEALLOCATE table_cursor;



Table: customers, Row Count: 500

Table: order_details, Row Count: 8981

Table: orders, Row Count: 3000

Table: products, Row Count: 200

Total execution time: 00:00:00.032